# 帮手程序说明

这一节，我们暂停推导公式的步伐，先了解一下自定义的帮手程序，以及 Hacking 帮手程序的一些技巧和一些程序问题．这可以看作是一份非正式的程序文档．

尽管与后面的公式推导无关，但由于以后经常调用帮手程序，因此这一节仍然非常重要．

帮手程序的路径应当在 `PYTHONPATH` 中．

## Jupyter 笔记本的库载入

默认下，Jupyter 笔记本的库不重复载入．这意味着在导入 (import) 库之后，如果你更改了库文件的代码，或者通过一些手段更改库文件的配置，你必须重启当前 Jupyter 笔记本的内核 (kernel) 以更新这些库的调用．

但如果执行下述代码，你可以通过再次导入库来更新库的调用，避免重启内核．这在用 Jupyter 调试代码的过程中会非常方便．

In [ ]:
%load_ext autoreload
%autoreload 2

## 帮手程序 utilities

In [ ]:
from utilities import val_from_fchk, timing, timing_level

### `val_from_fchk`

这个小程序可以从由 formchk 生成的文件中，通过传入数值的名称 (譬如 `Cartesian Gradient`)，返回其对应的数值．这个数值可以是一个值 (譬如能量)，也可以是一个向量 (压平后的梯度、Hessian 等)．我们在以前核验 Gaussian 与 PySCF 的 RHF 与 B3LYP 能量时已经使用过该函数．

### `timing_level`

这个小程序被用于对函数作挂墙时间 (wall time) 测评；挂墙时间与实际 CPU 运行时间 (CPU time) 不同．这个函数可以用作修饰符，这在 [Python 技巧](python_intro.ipynb#修饰符) 中已经有所介绍．

该函数可以传入一个数值参数，该数值参数是输出级别．若参数的值越低，那么程序越有可能输出挂墙时间．